# Word Embeddings: Encoding Lexical Semantics

https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

In [3]:
word_2_idx={"hello":0,"world":1}
embeds=nn.Embedding(2,5) # 2 words in vocab, 5 dimensional embeddings
lookup_tensor=torch.tensor([word_2_idx["hello"]],dtype=torch.long)
hello_embed=embeds(lookup_tensor)
hello_embed

tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229]],
       grad_fn=<EmbeddingBackward0>)

In [4]:
hello_embed

tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229]],
       grad_fn=<EmbeddingBackward0>)

In [5]:
lookup_tensor

tensor([0])

embeds = nn.Embedding(2, 5)  # 2 words in vocabulary, each with 5 dimensions  
The embedding layer here **does not know** and **does not care** about the words in the dictionary ; it just knows it has 2 indices 0 and 1, that it will learn embeddings for..  
The mapping word_2_idx = {"hello": 0, "world": 1} is done by you, you decide hello is associated with 0 and world with 1  


- The connection between "hello" and index 0 is made **outside the embedding layer** , in the word_2_idx dict  
- So, Row 0 doesnt inherently correspond to the word helo inside the embedding layer - it only corresponds to hello because you have manually mapped "hello" to index 0 with word_2_idx dict

- Initially, the embeddings are just random numbers with no inherent meaning .
- The embedding layer doesnt understand the meaning of "hello" or "world" right away. It starts with random numbers and learns meaningful embeddings **during training** :)
- During training, (with models like **Word2Vec**,**CBOW**) the model learns to adjust these embeddings based on the relationships in the training data

### N-gram language modeling

In [6]:
context_size=2
embedding_dim=10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

ngrams=[
    (
        [test_sentence[i-j-1] for j in range(context_size)],
        test_sentence[i]
    )

    for i in range(context_size,len(test_sentence))

]

print(ngrams[:3])

vocab=set(test_sentence)
word_2_idx={word:i for i,word in enumerate(vocab)}

class NGramLanguageModeler(nn.Module):
    def __init__(self,vocab_size,embedding_dim,context_size):
        super().__init__()
        self.embeddings=nn.Embedding(vocab_size,embedding_dim)
        self.linear1=nn.Linear(context_size*embedding_dim,128)
        self.linear2=nn.Linear(128,vocab_size)
    
    def forward(self,inputs):
        embeds=self.embeddings(inputs).view((1,-1))
        out=F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), embedding_dim, context_size)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_2_idx[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_2_idx[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_2_idx["beauty"]])


[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]
[519.7147989273071, 517.6384611129761, 515.5738348960876, 513.5212025642395, 511.4800395965576, 509.4487085342407, 507.42717242240906, 505.41412138938904, 503.4089243412018, 501.4102556705475]
tensor([-0.1563,  1.1377, -0.2516,  1.6710,  1.8876, -0.7539, -1.7129, -2.2062,
         0.3658, -0.4012], grad_fn=<SelectBackward0>)


## Computing Word Embeddings: Continuous Bag-of-Words

In [15]:
context_size=2
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

vocab=set(raw_text)
vocab_size=len(vocab)

word_2_idx={word:i for i,word in enumerate(vocab)}

data=[]

for i in range(context_size,len(raw_text)-context_size):
    context=(
        [raw_text[i-j-1]for j in range(context_size)] + [raw_text[i+j+1] for j in range(context_size)]
    )

    target=raw_text[i]
    data.append((context,target))

print(data[:5])
class CBOW(nn.Module):
    def __init__(self,vocab_size,embedding_dim):
        super().__init__()
        self.embeddings=nn.Embedding(vocab_size,embedding_dim) # W1 or W_context
        self.linear=nn.Linear(embedding_dim,vocab_size) # W_2 or W_word
    
    def forward(self,inputs):
        embedded=self.embeddings(inputs) #W_context(input) = h
        #mean
        h=embedded.mean(dim=0)
        out= self.linear(h) # W_word(h)
        log_probs = F.log_softmax(out.unsqueeze(0), dim=1)
        return log_probs


def make_context_vector(context,word_2_idx):
    idxs=[word_2_idx[w] for w in context]
    return torch.tensor(idxs,dtype=torch.long)


embedding_dim=10
learning_rate=0.01


losses=[]
loss_fn=nn.NLLLoss()
model=CBOW(vocab_size,embedding_dim)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


for epoch in range(100):
    total_loss=0

    for context,target in data:
        context_vector=make_context_vector(context,word_2_idx)
        target_idx=torch.tensor([word_2_idx[target]],dtype=torch.long)

        optimizer.zero_grad()

        log_probs=model(context_vector)
        loss=loss_fn(log_probs,target_idx)

        loss.backward()
        optimizer.step()
        total_loss+=loss.item()

    losses.append(total_loss)
print(losses)


context_vector = make_context_vector(data[0][0], word_2_idx)
output = model(context_vector)
print(model.embeddings.weight[word_2_idx["spirits"]])



# Example prediction check after training
context = ["a", "process"]  # Example context for prediction
context_vector = make_context_vector(context, word_2_idx)
model.eval()
with torch.no_grad():
    output = model(context_vector)
    predicted_index = torch.argmax(output).item()
    predicted_word = list(word_2_idx.keys())[predicted_index]
    print(f"Predicted word for context {context}: {predicted_word}")


# Check the embedding for a specific word
word_to_check = "spirits"
print(f"Embedding for '{word_to_check}':", model.embeddings.weight[word_2_idx[word_to_check]])
        



[(['are', 'We', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]
[230.11812114715576, 228.40634560585022, 226.7197527885437, 225.05737447738647, 223.41830158233643, 221.80166816711426, 220.20665979385376, 218.6325056552887, 217.0784628391266, 215.54383444786072, 214.0279405117035, 212.53013849258423, 211.04980731010437, 209.58635067939758, 208.1391944885254, 206.70777940750122, 205.29157423973083, 203.8900547027588, 202.50272417068481, 201.12909531593323, 199.76869869232178, 198.42108130455017, 197.08579683303833, 195.76242399215698, 194.4505476951599, 193.14976859092712, 191.8596968650818, 190.5799629688263, 189.31020283699036, 188.05006504058838, 186.79921913146973, 185.55733728408813, 184.32410621643066, 183.0992271900177, 181.88240790367126, 180.6733739376068, 179.47185623645782, 178.27760219573975, 177.09036469459534, 175.90991342067719, 174.